In [35]:
!pip install tree_sitter -qqq
!git clone https://github.com/BonaBeavis/tree-sitter-sparql

fatal: destination path 'tree-sitter-sparql' already exists and is not an empty directory.


In [36]:
from tree_sitter import Language, Parser

Language.build_library(
  'build/my-languages.so',
  ['tree-sitter-sparql']
)

SPARQL_LANGUAGE = Language('build/my-languages.so', 'sparql')
parser = Parser()
parser.set_language(SPARQL_LANGUAGE)

In [37]:
code = 'PREFIX foaf:  <http://xmlns.com/foaf/0.1/> SELECT ?name WHERE { ?person foaf:name ?name . }'

In [38]:
tree = parser.parse(bytes(code, "utf8"))

In [39]:
tree.root_node.children

[<Node kind=prologue, start_point=(0, 0), end_point=(0, 42)>,
 <Node kind=select_query, start_point=(0, 43), end_point=(0, 91)>]

In [40]:
tree.root_node.children[1].children

[<Node kind=select_clause, start_point=(0, 43), end_point=(0, 55)>,
 <Node kind=where_clause, start_point=(0, 56), end_point=(0, 91)>]

In [41]:
def get_string_from_code(node, lines):
  line_start = node.start_point[0]
  line_end = node.end_point[0]
  char_start = node.start_point[1]
  char_end = node.end_point[1]
  if line_start != line_end:
    code_list.append(' '.join([lines[line_start][char_start:]] + lines[line_start+1:line_end] + [lines[line_end][:char_end]]))
  else:
    code_list.append(lines[line_start][char_start:char_end])

def my_traverse(node, code_list):
  lines = code.split('\n')
  if node.child_count == 0:
    get_string_from_code(node, lines)
  elif node.type == 'string':
    get_string_from_code(node, lines)
  else:
    for n in node.children:
      my_traverse(n, code_list)
 
  return ' '.join(code_list)

In [42]:
code_list=[]
tokenized_code = my_traverse(tree.root_node, code_list)
print("Output after tokenization: " + tokenized_code)

Output after tokenization: PREFIX foaf : < > SELECT ?name WHERE { ?person foaf : name ?name . }


In [43]:
code = 'SELECT ?name WHERE { ?person foaf:name ?name . }'
tree = parser.parse(bytes(code, "utf8"))
code_list=[]
tokenized_code = my_traverse(tree.root_node, code_list)
print("Output after tokenization: " + tokenized_code)

Output after tokenization: SELECT ?name WHERE { ?person foaf : name ?name . }
